# Sobre o Conjunto de Dados

### Modelagem ML para identificar a relação de genomas com os genes ortólogos

Analise do dataset com os outcomes (0 não patógenos e 1 patógeno). 
As variáveis (OG grupos de genes ortólogos) estão nas colunas e nas linhas estão as instâncias (genomas). 
Inicialmente tente descobrir os OG envolvidos na patogênese, os módulos funcionais (os OG que estão atuando em conjunto).

Dicas: A presença do gene é mais importante do que ausências! Ou seja, o(s) gene(s) precisam estar lá para desempenhar a interação com o hospedeiro e a função da patogênese. 
Alguns não patógenos podem ter alguns poucos genes de virulência também
mas é preciso ter a grande maioria deles para ser um patógeno eficiente de importância clínica.

In [1]:
import os
import gzip
import pandas as pd
import numpy as np

In [2]:
with gzip.open("C:\\Users\\wesle\\Projetos_Estudo_Analise-de-Dados\\Notebooks\\ML_Dashboard\\dataset.csv.gz", 'rb') as f:
    df = pd.read_csv(f,sep=',').sample(10000)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
df.rename(columns= {"GENOME" :"Genome"}, inplace = True)

In [ ]:
df['Target'] = df['Target'].apply(lambda x : 1 if(x == True) else 0)

In [ ]:
df.set_index("Genome", inplace = True)

### Analise de Frequencias 

In [ ]:
df['Freq'] = ((df['Target']/df.groupby('Target')['Target'].transform('sum'))*100).cumsum()

In [ ]:
df.sort_values(by='Freq', ascending=False, inplace = True)

In [ ]:
df_freq = df[df['Freq'] >= 25.0]

In [ ]:
df_freq

## Modelo de machine Learning

### Vamos começar a preparar os dados para testar um modelo de classificação

### Separe o conjunto de dados em dois grupos: treino e teste nas proporções de 70% e 30% respectivamente.

* Lembre-se que primeiro vai ser preciso separar as features do target antes de usar o train_test_split. Nosso target é a coluna 'Status'.
* Estamos lidando com dados desbalanceados, então pode ser legal setar o argumento stratify do train_test_split usando a variável target.

In [ ]:
# Indicando as variáveis “x” e “y” estamos dizendo que queremos separar estas duas variáveis em dois grupos, sendo que com o parâmetro “test_size = 0.3” 
# determinamos que os dados de teste receberão 30% dos dados, e os dados de treino 70%. Como estamos dividindo duas bases
# como resultado teremos quatro bases, sendo que cada uma será salva na respectiva variável.

from sklearn.model_selection import train_test_split

# Criamos então a variável “y” com os dados da coluna “target”, ou seja, os dados que iremos prever. Criamos também a variável “x” com as demais colunas do dataset, que são as variáveis preditoras.

x = df_freq.drop(columns = ["Target", 'Freq'], axis = 1)
y = df_freq['Target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4)

print(x_train.shape, 
      x_test.shape, 
      y_train.shape,
      y_test.shape)

## HeatMap de Correlação 

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sn

In [ ]:
df_train = pd.concat([x_train, y_train], axis=1)
print(df_train.shape)
df_train.head()

In [ ]:
df_train

In [ ]:
plt.figure(figsize=(15, 15))
sn.heatmap(df_train.corr(), annot=True, cmap='magma');

### Plot SHAP

In [ ]:
import shap

# load JS visualization code to notebook
shap.initjs()

### Agora vamos treinar uma Árvore de Decisão nos dados de treino
* Comece com uma primeira versão definindo apenas o parâmetro max_depth = 3
* Dica: Não é preciso reescalonar as variáveis numéricas porque estamos usando uma Decision Tree.

In [ ]:
# ExtraTrees, que criará várias árvores de decisão
# Este algoritmo está pronto para ser utilizado 
# sendo que precisamos apenas indicar sua função
# Como estamos trabalhando em um problema de classificação utilizaremos a função “ExtraTreesClassifier()”.

from sklearn.ensemble import ExtraTreesClassifier

model_decisiontree = ExtraTreesClassifier()

model_decisiontree.fit(x_train,y_train)

acc_decisiontree = round(model_decisiontree.score(x_test, y_test)*100,2)

acc_decisiontree

In [ ]:
explainer_model_decisiontree = shap.TreeExplainer(model_decisiontree)
shap_test_model_decisiontree = explainer_model_decisiontree.shap_values(x_test)

In [ ]:
shap.force_plot(explainer_model_decisiontree.expected_value, shap_test_model_decisiontree, x_test)

In [ ]:
shap.summary_plot(shap_test_model_decisiontree)

## Random Forest Regressor

In [ ]:
# Uma floresta aleatória é um meta estimador que ajusta uma série de árvores de decisão de classificação em várias subamostras do conjunto de dados 
# e usa a média para melhorar a precisão preditiva e controlar o ajuste excessivo

#Train the model
from sklearn.ensemble import RandomForestRegressor

model_RandomForestRegressor = RandomForestRegressor(n_estimators=100)
#Fit

model_RandomForestRegressor.fit(x_train,y_train)

#Score/Accuracy
acc_randomforest=round(model_RandomForestRegressor.score(x_test, y_test)*100,2)

acc_randomforest

In [ ]:
explainer_model_RandomForestRegressor = shap.TreeExplainer(model_RandomForestRegressor)
shap_test_model_RandomForestRegressor = explainer_model_RandomForestRegressor.shap_values(x_test)

In [ ]:
shap.force_plot(explainer_model_RandomForestRegressor.expected_value[1], shap_test_model_RandomForestRegressor[][0,:], x_test.iloc[0,:])

In [ ]:
shap.summary_plot(shap_test_model_decisiontree[1], x_test)

In [ ]:
# explainer = shap.Explainer(model_RandomForestRegressor)
# shap_test = explainer(x_test)
# shap.summary_plot(shap_test)
# print(f"Shap values length: {len(shap_test)}\n")
# print(f"Sample shap value:\n{shap_test[0]}")

## Regressão Logistica

In [ ]:
# from sklearn.linear_model import LogisticRegression

# model_LogisticRegression = LogisticRegression()

# #Fit the model
# model_LogisticRegression.fit(x_train,y_train)

# y_pred = model_LogisticRegression.predict(x_test)

# #Score/Accuracy
# acc_logreg=round(model_LogisticRegression.score(x_test, y_test)*100,2)

# acc_logreg

In [ ]:
explainer_model_LogisticRegression = shap.LinearExplainer(model_LogisticRegression, x_train, feature_dependence="independent")
shap_test_model_LogisticRegression = explainer_model_LogisticRegression(x_test)

In [ ]:
shap.summary_plot(shap_test_model_decisiontree[1], x_test)

## XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import pickle
import warnings

# df_train = pd.read_csv("data/dataset.csv").sample(10000)
xgb = XGBClassifier()
xgb.fit(x, y)
xgb_scores = cross_val_score(xgb, x, y, cv=10, scoring='roc_auc')
xgb_score = np.mean(xgb_scores)
print("XGB - AUC (ROC): ", xgb_score)